In [ ]:
import json

# given the output of our RAG system
with open("../data/evaluation_results_2025-04-04_11-08-43.jsonl", "r") as file:
    data = [json.loads(line) for line in file]

In [ ]:
# Given a patient with symptoms
query = data[0]

# Information about the patient
general_demographics = query["general_demographics"]
symptoms_description = query["symptoms_description"]

# Informationa about the chunks retrieved (to be used when reasoning)
k = query["k"]
retrieved_chunks = query["retrieved_documents"]
retrieved_documents_scores = query["retrieved_documents_scores"]
retrieved_documents_sources = query["retrieved_documents_sources"]

# Grouping information about the chunks retrieved by the potential condition
potential_conditions = {x: [] for x in set(retrieved_documents_sources)}

for i in range(len(retrieved_documents_sources)):
    potential_conditions[retrieved_documents_sources[i]].append((retrieved_chunks[i]))

print(potential_conditions.keys())

# And also information for measuring performance (not to use when querying the model)
query_type = query["query_type"]
severity_level = query["severity_level"]
conditions_title = query["conditions_title"]

In [ ]:
import ollama


def get_response_from_model(prompt, model="gemma3:1b"):
    response = ollama.generate(model=model, prompt=prompt)
    return response["response"]

In [ ]:
prompt_template = f"""
Given this symptom description: "{symptoms_description}".
And the following information about the patient: {general_demographics}.
Our system suggests one of the following conditions: {", ".join(potential_conditions.keys())}.

Decide which of the following conditions is the most likely and the level of severity among the following options:

### Severity Level:
* **Not urgent**: Monitor at home
* **Medium**: Go to the GP
* **Medium urgent**: Get an urgent GP appointment
* **Urgent**: Call 999

For each potential condition, we provide the most relevant information from the retrieved documents:

{json.dumps(potential_conditions, indent=2)}

Reply only with the most likely condition and the severity level, in the following format:

### Condition: <condition>
### Severity Level: <severity_level>

If you think that the condition is not listed, please say "None of the above".
"""

In [ ]:
print(prompt_template)

In [ ]:
response = get_response_from_model(prompt_template, model="deepseek-r1:8b")
print(response)